In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.8MB 7.3MB/s eta 0:00:01     |████▋                           | 829kB 7.3MB/s eta 0:00:01     |████████▏                       | 1.5MB 7.3MB/s eta 0:00:01     |████████████████████████        | 4.3MB 7.3MB/s eta 0:00:01     |███████████████████████████▋    | 5.0MB 7.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 

In [ ]:
#from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import requests
import lxml
from sklearn.cluster import KMeans
from sklearn import metrics
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

In [ ]:
!wget -q -O 'berlin_data.csv' https://raw.githubusercontent.com/zauberware/postal-codes-json-xml-csv/240d643985f3654918665721d47e2642112d81e6/data/DE/zipcodes.de.csv
coordinates_df = pd.read_csv('berlin_data.csv')

In [ ]:
coordinates_df = coordinates_df[coordinates_df.state != 'Bayern']
coordinates_df = coordinates_df[coordinates_df.state != 'Bremen']
coordinates_df = coordinates_df[coordinates_df.state != 'Hessen']
coordinates_df = coordinates_df[coordinates_df.state != 'Baden-Württemberg']
coordinates_df = coordinates_df[coordinates_df.state != 'Hamburg']
coordinates_df = coordinates_df[coordinates_df.state != 'Mecklenburg-Vorpommern']
coordinates_df = coordinates_df[coordinates_df.state != 'Niedersachsen']
coordinates_df = coordinates_df[coordinates_df.state != 'Nordrhein-Westfalen']
coordinates_df = coordinates_df[coordinates_df.state != 'Rheinland-Pfalz']
coordinates_df = coordinates_df[coordinates_df.state != 'Schleswig-Holstein']
coordinates_df = coordinates_df[coordinates_df.state != 'Saarland']
coordinates_df = coordinates_df[coordinates_df.state != 'Sachsen-Anhalt']
coordinates_df = coordinates_df[coordinates_df.state != 'Sachsen']
coordinates_df = coordinates_df[coordinates_df.state != 'Thüringen']
coordinates_df = coordinates_df[coordinates_df.state != 'Brandenburg']
coordinates_df.rename(columns={'zipcode': 'Postal code', 'latitude': 'Latitude', 'longitude': 'Longitude'}, inplace=True)
coordinates_df = coordinates_df.drop(['country_code', 'state_code', 'province', 'province_code', 'community', 'community_code', 'state'], axis =1)            

In [ ]:
coordinates_df.head()

In [ ]:
map_berlin = folium.Map(location=[52.5323, 13.3846], zoom_start=11)
for lat, lng, label in zip(coordinates_df['Latitude'], coordinates_df['Longitude'], coordinates_df['Postal code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)

In [ ]:
CLIENT_ID = 'JABYZYQHJAAA1VYXYMDSTJRVRAVRXP0K51YH41BV2JBY5EBO' # your Foursquare ID
CLIENT_SECRET = 'NUHB32I5UVYXUZG2EA5YVWMNKX1YLNYVDNSE0VQPZGRVF0YU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50
radius = 2000

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal code', 
                  'Postal code Latitude', 
                  'Postal code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
berlin_venues = getNearbyVenues(names=coordinates_df['Postal code'],
                                   latitudes=coordinates_df['Latitude'],
                                   longitudes=coordinates_df['Longitude']
                                  )

In [ ]:
berlin_venues.to_csv("berlin_venues.csv",index=False, encoding='utf8')

In [ ]:
berlin_onehot = pd.get_dummies(berlin_venues[['Venue Category']], prefix="", prefix_sep="") # one hot encoding

berlin_onehot['Postal code'] = berlin_venues['Postal code'] # add Postal code column back to dataframe

fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1]) # move Postal code column to the first column
berlin_onehot = berlin_onehot[fixed_columns]

berlin_grouped = berlin_onehot.groupby('Postal code').mean().reset_index() # group rows by Postal code and by taking the mean of the frequency of occurrence of each category

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# print each Postal code along with the top 5 most common venues
num_top_venues = 50

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Postal_code_venues_sorted = pd.DataFrame(columns=columns)
Postal_code_venues_sorted['Postal code'] = berlin_grouped['Postal code']

for ind in np.arange(berlin_grouped.shape[0]):
    Postal_code_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)

In [ ]:
# using an elbow plot to see at what value of k, the distance between the mean of a cluster and the other data points in the cluster is at its lowest.
sum_of_squared_distances = []
K = range(1,30)
berlin_grouped_clustering = berlin_grouped.drop('Postal code', 1)
for k in K:
    k_means = KMeans(n_clusters=k)
    model = k_means.fit(berlin_grouped_clustering)
    sum_of_squared_distances.append(k_means.inertia_)
    
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('sum_of_squared_distances')
plt.title('elbow method for optimal k')
plt.show()

In [ ]:
# Cluster the Postal codes

# set number of clusters
kclusters = 23

berlin_grouped_clustering = berlin_grouped.drop('Postal code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
Postal_code_venues_sorted.head()

In [ ]:
#  create a new dataframe that includes the cluster as well as the top 10 venues for each Postal code

# add clustering labels
Postal_code_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

berlin_merged = coordinates_df

# merge berlin_grouped with berlin_data to add latitude/longitude for each Postal code
berlin_merged = berlin_merged.join(Postal_code_venues_sorted.set_index('Postal code'), on='Postal code')
berlin_merged = berlin_merged.dropna()
berlin_merged = berlin_merged.astype({'ClusterLabels': int})

berlin_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[52.5323, 13.3846], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(berlin_merged['Latitude'], berlin_merged['Longitude'], berlin_merged['Postal code'], berlin_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
berlin_merged.to_csv("output_filename.csv", index=False, encoding='utf8')